<a href="https://colab.research.google.com/github/Hydrometeorological-Remote-Sensing/cropmapping/blob/sw_ww_test/predict_pytorch_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q segmentation_models_pytorch 
!pip install -q affine
!pip install -q rioxarray 

In [2]:
import datetime
import segmentation_models_pytorch as smp
import numpy as np
import torch
from glob import glob
import torch.utils.data
from data_loader import data_load
import tensorflow as tf
from unet_pytorch import build_unet
import torch.nn as nn
import json
import xarray as xr
from torch.utils.tensorboard import SummaryWriter
from copy import deepcopy
import pandas as pd
from affine import Affine
from scipy.sparse import *
import rioxarray

2023-03-06 12:30:57.184666: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 12:30:57.369546: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-06 12:30:58.104901: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-06 12:30:58.104974: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [3]:
def to_image(img, mixer, raster_dir):
    f = open(*mixer)
    mixer = json.load(f)
    f.close()
    doubleMatrix = mixer["projection"]["affine"]["doubleMatrix"]
    patchesPerRow = mixer["totalPatches"] / mixer["patchesPerRow"]
    img = np.split(img, patchesPerRow)
    img = [np.hstack(l) for l in img]
    img = np.vstack(img).astype("int8")
    ds = xr.DataArray(img)
    img = []
    ds = ds.rio.write_crs(4326, inplace=True)
    transform = Affine(*doubleMatrix)
    ds.rio.write_transform(transform, inplace=True)
    ds.spatial_ref.GeoTransform
    ds.rio.set_spatial_dims("dim_1", "dim_0")
    ds.rio.to_raster(raster_dir)

In [4]:
!mkdir npyfiles

mkdir: cannot create directory ‘npyfiles’: File exists


In [10]:
# from state_dict import *
def doPrediction(tf_files, model,bands,description,batch_size,device,x_buffer,y_buffer,kernel_shape):
    # print(tf_files)
    file_ls = np.append(
        np.arange(0, 70) * int(len(list(tf_files)) / 70), len(list(tf_files))
    )
    patches = 0
    np_ls = []
    _ = 1
    for i in range(len(file_ls) - 1):
        print(i)
        data = data_load(
                            tf_files[file_ls[i] : file_ls[i + 1]],
                            bands,
                            description,
                        ).get_pridiction_dataset().batch(batch_size)

        ls = []
        for batch in data.as_numpy_iterator():
            # print(batch.shape)
            predictions = torch.tensor(batch).to(device)
            with torch.no_grad():
                predictions = model(predictions).cpu().numpy()  # .tolist()
            # print(predictions.shape)
            print(predictions)
            predictions = predictions.argmax(axis=1).squeeze()[
                :,
                x_buffer : x_buffer + kernel_shape[0],
                y_buffer : y_buffer + kernel_shape[1],
            ]
            print("Writing patch " + str(_ * 64) + "...", flush=True)
            _ = _ + 1
            ls.append(predictions)
        np_ls.append(np.vstack(ls).astype("int8"))
    return np_ls
    print("finished")

In [8]:
ckpt_path = "/bess23/huaize/semi-supervised/models/best_03_06_00_37_03.pth"

bands = [
    "blue",
    "green",
    "red",
    "nir",
    "swir1",
    "swir2",
    "ndvi",
    "nirv"
]
targets = ["cropland"]
features = bands  # + targets
nclass = 2
batch_size = 256
class_names = ["Others", "Wheat"]
kernel_shape = [256, 256]
kernel_buffer = [128, 128]
# Get set up for prediction.
x_buffer = int(kernel_buffer[0] / 2)
y_buffer = int(kernel_buffer[1] / 2)

buffered_shape = [
    kernel_shape[0] + kernel_buffer[0],
    kernel_shape[1] + kernel_buffer[1],
]
columns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) for k in features
]
description = dict(zip(features, columns))
metrics_dict = {}
device = "cuda"
folder = "prediction"
model = (
    build_unet(len(bands), nclass).cuda()
    if device == "cuda"
    else build_unet(len(bands), nclass)
)

model.load_state_dict(torch.load(ckpt_path))
model = nn.DataParallel(model)
model.eval()

DataParallel(
  (module): build_unet(
    (e1): encoder_block(
      (conv): conv_block(
        (conv1): Conv2d(8, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    )
    (e2): encoder_block(
      (conv): conv_block(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [11]:
states = ['North Dakota'] 
for state in states:
    print("predicting ", state)
    folder = (
        "/bess23/huaize/semi-supervised/data/unlabeled"
        )
    tf_files = glob(folder + "/*.gz")
    tf_files.sort()
    json_file = glob(folder + "/*.json")
    out_image_file = "/bess23/huaize/semi-supervised/tif/{}_pytorch.tif".format(
        state
    )
    prediction = doPrediction(tf_files, model,bands,description,batch_size,device,x_buffer,y_buffer,kernel_shape)
    # tifname = state
    # files = sorted(glob(f"npyfiles/*.npy"), key=os.path.getmtime)
    # img = np.vstack(prediction)
    # # np.save("/content/drive/MyDrive/RDA/geotiff_result/{}.npy".format(state), img)
    # to_image(img, json_file, out_image_file)

predicting  North Dakota
0
[[[[0.5433501  0.54498696 0.54555184 ... 0.5457241  0.54532087
    0.5444545 ]
   [0.54338104 0.5457342  0.54556    ... 0.5461302  0.5447308
    0.5441512 ]
   [0.5442228  0.54637897 0.5465878  ... 0.54671824 0.54522115
    0.54419464]
   ...
   [0.5439824  0.5464573  0.546148   ... 0.5490299  0.546303
    0.5453928 ]
   [0.5437384  0.5454239  0.545463   ... 0.54833096 0.5462358
    0.54480654]
   [0.5416143  0.5427146  0.5424049  ... 0.5443199  0.5426409
    0.54264754]]

  [[0.45664996 0.45501304 0.45444813 ... 0.45427585 0.45467907
    0.4555455 ]
   [0.4566189  0.45426577 0.45443997 ... 0.45386976 0.4552692
    0.4558488 ]
   [0.4557773  0.45362103 0.45341218 ... 0.45328176 0.45477885
    0.45580533]
   ...
   [0.4560176  0.45354274 0.45385197 ... 0.45097008 0.45369706
    0.45460722]
   [0.45626155 0.45457605 0.45453694 ... 0.4516691  0.45376414
    0.45519346]
   [0.45838574 0.4572854  0.4575951  ... 0.4556801  0.45735905
    0.45735243]]]


 [[[0.54335

KeyboardInterrupt: 

In [13]:
prediction

[array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        ...,
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],